In [1]:
from lxml import etree, html
from lxml.html.clean import clean_html
import pandas as pd
from datetime import date, datetime
import requests
import re
import os
import shutil

# Mueve todos los fichero del directorio de trabajo a otro de archivo

In [2]:
source_dir = './BOEs'
target_dir = './BOEs_Anteriores'
    
file_names = os.listdir(source_dir)
    
for file_name in file_names:
    #shutil.move(os.path.join(source_dir, file_name), target_dir)
    # Evita que de error si el fichero que se mueve ya existe en dir destino
    try:
        os.remove(os.path.join(target_dir, file_name))
        shutil.move(os.path.join(source_dir, file_name), target_dir)
    except OSError:
        shutil.move(os.path.join(source_dir, file_name), target_dir)

# Recoge resumen diario del BOE de hoy

In [3]:

today = date.today()

# dd/mm/YYYY
hoy = today.strftime("%Y%m%d")
print("Fecha de Hoy =", hoy)

# dd
d = today.strftime("%d")
print("dia =", d)

# mm
m = today.strftime("%m")
print("mes =", m)

# YYYY
Y = today.strftime("%Y")
print("año =", Y)

print(today.strftime("%d/%m/%Y"))

Fecha de Hoy = 20210218
dia = 18
mes = 02
año = 2021
18/02/2021


In [4]:
URL_XML_resumen =  "https://www.boe.es/diario_boe/xml.php?id=BOE-S-" + str(hoy)

In [5]:
URL_XML_resumen

'https://www.boe.es/diario_boe/xml.php?id=BOE-S-20210218'

In [6]:
url = URL_XML_resumen
r = requests.get(url)

In [7]:
def save_html(html, path):
    with open(path, 'wb') as f:
        f.write(html)

save_html(r.content, './BOEs/Resumen-BOE-' + hoy + '.xml')

In [8]:
resumen = etree.parse('./BOEs/Resumen-BOE-' + hoy + '.xml')

In [9]:
raiz=resumen.getroot()
raiz.tag
raiz_sumario = raiz

In [10]:
seccion = raiz.findall("sumario/diario/seccion")
seccion

[]

In [11]:
for seccion in raiz.xpath('//seccion'):
    nombre_seccion = seccion.xpath('@nombre')
    print(nombre_seccion)

['I. Disposiciones generales']
['II. Autoridades y personal. - A. Nombramientos, situaciones e incidencias']
['II. Autoridades y personal. - B. Oposiciones y concursos']
['III. Otras disposiciones']
['IV. Administración de Justicia']
['V. Anuncios. - A. Contratación del Sector Público']
['V. Anuncios. - B. Otros anuncios oficiales']
['V. Anuncios. - C. Anuncios particulares']


In [12]:
secciones = raiz.findall("sumario/diario/seccion")
for seccion in secciones:
    print(seccion.text)

In [13]:
tabla_resumen = pd.DataFrame()

In [14]:
for seccion in raiz.xpath('//seccion[contains(@nombre, "I. Disposiciones generales")]'):
    # el = doc.xpath("//div[contains(@class, 'channel') and not(contains(@class, 'disabled'))]")
    nombre_seccion = seccion.xpath('@nombre')

    for departamento in seccion:
        nombre_departamento = departamento.xpath('@nombre') 

        for epigrafe in departamento:
            nombre_epigrafe = epigrafe.xpath('@nombre')

            for item in epigrafe:
                item_id = item.xpath('@id')
                item_name = item.xpath('.//titulo/text()') 

                item_urlXml = "https://www.boe.es" + str(item.xpath('.//urlXml/text()'))[2:-2]

                tabla_resumen = tabla_resumen.append({'Seccion': nombre_seccion, 
                                                      'Departamento': nombre_departamento, 
                                                      'Epigrafe' : nombre_epigrafe,
                                                      'Item_id': item_id, 
                                                      'Item_Nombre' : item_name, 
                                                      'Item_URL_XML' : item_urlXml},
                                                      ignore_index=True)


In [15]:
tabla_resumen.sort_values('Item_id')

,Departamento,Epigrafe,Item_Nombre,Item_URL_XML,Item_id,Seccion
0,[JEFATURA DEL ESTADO],[Medidas urgentes],[Corrección de errores del Real Decreto-ley 3/...,https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-2453],[I. Disposiciones generales]
1,"[MINISTERIO DE ASUNTOS EXTERIORES, UNIÓN EUROP...",[Acuerdos internacionales administrativos],[Acta de la VIII Reunión de la Comisión Mixta ...,https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-2454],[I. Disposiciones generales]
2,"[MINISTERIO DE ASUNTOS EXTERIORES, UNIÓN EUROP...",[Acuerdos internacionales administrativos],[Acuerdo entre la Organización Mundial del Tur...,https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-2455],[I. Disposiciones generales]
3,"[MINISTERIO DE ASUNTOS EXTERIORES, UNIÓN EUROP...",[Acuerdos internacionales administrativos],[Acuerdo Internacional Administrativo entre la...,https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-2456],[I. Disposiciones generales]
4,[MINISTERIO DE HACIENDA],[Corporaciones locales. Información tributaria],"[Resolución de 10 de febrero de 2021, de la Se...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-2457],[I. Disposiciones generales]
5,[MINISTERIO DE SANIDAD],[Medidas sanitarias],"[Orden SND/133/2021, de 17 de febrero, sobre l...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-2458],[I. Disposiciones generales]


# Descarga ficheros XML asociados

In [16]:
for item_URL in tabla_resumen['Item_URL_XML']:
    r = requests.get(item_URL)
    #f = './BOEs/' + item_URL[-16:] + '.xml'
    ### Separa el número del BOE del resto de la cadena y aplica expresión REGEX 
    print(item_URL.split('='))
    print(re.match('BOE\-A\-[0-9]+\-[0-9]+',item_URL.split('=')[1]))
    filename = re.match('BOE\-A\-[0-9]+\-[0-9]+',item_URL.split('=')[1]).group()
    f = './BOEs/' + filename + '.xml'
    save_html(r.content, f)

['https://www.boe.es/diario_boe/xml.php?id', 'BOE-A-2021-2453']
<re.Match object; span=(0, 15), match='BOE-A-2021-2453'>
['https://www.boe.es/diario_boe/xml.php?id', 'BOE-A-2021-2454']
<re.Match object; span=(0, 15), match='BOE-A-2021-2454'>
['https://www.boe.es/diario_boe/xml.php?id', 'BOE-A-2021-2455']
<re.Match object; span=(0, 15), match='BOE-A-2021-2455'>
['https://www.boe.es/diario_boe/xml.php?id', 'BOE-A-2021-2456']
<re.Match object; span=(0, 15), match='BOE-A-2021-2456'>
['https://www.boe.es/diario_boe/xml.php?id', 'BOE-A-2021-2457']
<re.Match object; span=(0, 15), match='BOE-A-2021-2457'>
['https://www.boe.es/diario_boe/xml.php?id', 'BOE-A-2021-2458']
<re.Match object; span=(0, 15), match='BOE-A-2021-2458'>


# Importa de nuevo los XML generados

In [17]:

import glob
BOEs = glob.glob('./BOEs/BOE*.xml')
print (BOEs)

['./BOEs/BOE-A-2021-2454.xml', './BOEs/BOE-A-2021-2455.xml', './BOEs/BOE-A-2021-2457.xml', './BOEs/BOE-A-2021-2456.xml', './BOEs/BOE-A-2021-2453.xml', './BOEs/BOE-A-2021-2458.xml']


# Genera DF con datos Análisis de cada XML

In [18]:
tabla_analisis = pd.DataFrame()

for BOE in BOEs:
    #print (BOE)
    BOE_XML = etree.parse(BOE)
    raiz=BOE_XML.getroot()

    materias = [materia.text for materia in raiz.findall('analisis/materias/materia')]
    alertas = [alerta.text for alerta in raiz.findall('analisis/alertas/alerta')]
    
    referencias = [str(referencia.xpath('@referencia')).replace("['",'').replace("']",'') for referencia in raiz.findall('analisis/referencias/anteriores/anterior')]
    referencias_palabras = [referencia.text for referencia in raiz.findall('analisis/referencias/anteriores/anterior/palabra')]
    referencias_texto = [referencia.text for referencia in raiz.findall('analisis/referencias/anteriores/anterior/texto')]

    Item_Id = raiz.xpath(".//identificador/text()")
    Item_name = raiz.xpath('.//titulo/text()') 

    Fecha_Publicacion = datetime.strptime(str(raiz.xpath('.//fecha_publicacion/text()'))[2:-2],"%Y%m%d")
    #Fecha_Publicacion = str(raiz.xpath('.//fecha_publicacion/text()'))[2:-2]
 
    tabla_analisis = tabla_analisis.append({'Item_id': Item_Id,
                                            'Item_Name' : Item_name,
                                            'Fecha_publicacion' : Fecha_Publicacion,
                                            'Materias' : materias,
                                            'Alertas' : alertas,
                                            'Referencias' : referencias,
                                            'Referencias_palabra' : referencias_palabras,
                                            'Referencias_texto' : referencias_texto},
                                            ignore_index=True)


In [19]:
tabla_analisis.sort_values('Item_id')

,Alertas,Fecha_publicacion,Item_Name,Item_id,Materias,Referencias,Referencias_palabra,Referencias_texto
4,"[Asuntos sociales, Derecho Administrativo]",2021-02-18,[Corrección de errores del Real Decreto-ley 3/...,[BOE-A-2021-2453],"[Asistencia social, Ayudas, Registros administ...","[BOE-A-2021-1529, BOE-A-2020-5493]","[CORRECCIÓN de errores, CORRIGE errores]","[del Real Decreto-ley 3/2021, de 2 de febrero,..."
0,[Relaciones internacionales],2021-02-18,[Acta de la VIII Reunión de la Comisión Mixta ...,[BOE-A-2021-2454],"[Acuerdos internacionales, Ayuda al desarrollo...",[],[],[]
1,"[Relaciones internacionales, Turismo]",2021-02-18,[Acuerdo entre la Organización Mundial del Tur...,[BOE-A-2021-2455],"[Acuerdos internacionales, Organización Mundia...","[BOE-A-2015-8565, BOE-A-1974-1937]","[DE CONFORMIDAD con, DE CONFORMIDAD con]",[el art. 33 del Convenio de 25 de junio de 201...
3,"[Relaciones internacionales, Turismo]",2021-02-18,[Acuerdo Internacional Administrativo entre la...,[BOE-A-2021-2456],"[Acuerdos internacionales, Congresos, Madrid, ...",[BOE-A-2015-8565],[DE CONFORMIDAD con],[el Convenio de 25 de junio de 2015]
2,"[Organización de la Administración, Sistema tr...",2021-02-18,"[Resolución de 10 de febrero de 2021, de la Se...",[BOE-A-2021-2457],"[Administración Local, Ayuntamientos, Cesión d...",[BOE-A-2020-17339],[DE CONFORMIDAD con],"[el art. 95.3 de la Ley 11/2020, de 30 de dici..."
5,[],2021-02-18,"[Orden SND/133/2021, de 17 de febrero, sobre l...",[BOE-A-2021-2458],[],"[BOE-A-2011-15623, BOE-A-1986-10498, BOE-A-202...","[DE CONFORMIDAD con, DE CONFORMIDAD con, EN RE...","[el art. 52 de la Ley 33/2011, de 4 de octubre..."


In [20]:
# Crea nueva columna vacía de tipo lista en tabla_analisis
tabla_analisis['Referencias_completas'] = [[] for i in range(len(tabla_analisis))]
tabla_analisis['Tags'] = [[] for i in range(len(tabla_analisis))]
tabla_analisis['Match_ASECORP_BBDD'] = [[] for i in range(len(tabla_analisis))]


In [21]:
# Define expresiones REGEX para búsqueda de leyes, decretos, etc. referenciadas anteriormente
pattern = ['Ley [0-9]+\/[0-9]+','Real Decreto [0-9]+\/[0-9]+','Real Decreto Legislativo [0-9]+\/[0-9]+','Real Decreto-ley [0-9]+\/[0-9]+','Orden [A-Z]+\/[0-9]+\/[0-9]+','Orden Circular [0-9]+\/[0-9]+','Reglamento \(UE\) [0-9]+\/[0-9]+', 'Reglamento de Ejeución \(UE\) [0-9]+\/[0-9]+' ,'Sentencia de [0-9]+ de [a-z]+ de [0-9]+','Sentencia [0-9]+\/[0-9]+','Orden de [0-9]+ de [a-z]+ de [0-9]+', 'Resolución de [0-9]+ de [a-z]+ de [0-9]+','Resolución [a-z]+\/[0-9]+\/[0-9]+', 'Nota de Servicio [0-9]+\/[0-9]+', 'Acuerdo multilateral M\-[0-9]+', 'Circular [0-9]+\/[0-9]+', 'Decisión \(UE\) [0-9]+\/[0-9]+', 'Decisión de Ejecución \(UE\) [0-9]+\/[0-9]+']

In [22]:
# Consolida las columnas Referencias_palabra y Referencias_texto en una única frase
for i, row in tabla_analisis.iterrows():
    for item_list in range(len(row['Referencias'])): 
        row['Referencias_completas'].append(row['Referencias_palabra'][item_list] + ' ' + row['Referencias_texto'][item_list])
        tabla_analisis['Tags'][i] = re.findall('|'.join(pattern), str(row['Referencias_texto']), flags=re.IGNORECASE)
    print(row['Referencias_completas'])

[]
['DE CONFORMIDAD con el art. 33 del Convenio de 25 de junio de 2015', 'DE CONFORMIDAD con al art. 3 de los Estatutos de 27 de septiembre de 1970']
['DE CONFORMIDAD con el art. 95.3 de la Ley 11/2020, de 30 de diciembre']
['DE CONFORMIDAD con el Convenio de 25 de junio de 2015']
['CORRECCIÓN de errores del Real Decreto-ley 3/2021, de 2 de febrero', 'CORRIGE errores en la disposición transitoria 8 del Real Decreto-ley 20/2020, de 29 de mayo']
['DE CONFORMIDAD con el art. 52 de la Ley 33/2011, de 4 de octubre', 'DE CONFORMIDAD con los arts. 2 y 3 de la Ley Orgánica 3/1986, de 14 de abri', 'EN RELACIÓN con la Orden PCM/79/2021, de 2 de febrero', 'CITA Real Decreto 926/2020, de 25 de octubre', 'CITA Ley 29/1998, de 13 de julio', 'CITA Real Decreto 1418/1986, de 13 de junio']


In [23]:
# Elimina Tags duplicados
for i, row in tabla_analisis.iterrows():
    tabla_analisis['Tags'][i] = list(set(tabla_analisis['Tags'][i]))
    print(tabla_analisis['Tags'][i])

[]
[]
['Ley 11/2020']
[]
['Real Decreto-ley 3/2021', 'Real Decreto-ley 20/2020']
['Real Decreto 1418/1986', 'Real Decreto 926/2020', 'Ley 29/1998', 'Ley 33/2011', 'Orden PCM/79/2021']


In [24]:
tabla_analisis

,Alertas,Fecha_publicacion,Item_Name,Item_id,Materias,Referencias,Referencias_palabra,Referencias_texto,Referencias_completas,Tags,Match_ASECORP_BBDD
0,[Relaciones internacionales],2021-02-18,[Acta de la VIII Reunión de la Comisión Mixta ...,[BOE-A-2021-2454],"[Acuerdos internacionales, Ayuda al desarrollo...",[],[],[],[],[],[]
1,"[Relaciones internacionales, Turismo]",2021-02-18,[Acuerdo entre la Organización Mundial del Tur...,[BOE-A-2021-2455],"[Acuerdos internacionales, Organización Mundia...","[BOE-A-2015-8565, BOE-A-1974-1937]","[DE CONFORMIDAD con, DE CONFORMIDAD con]",[el art. 33 del Convenio de 25 de junio de 201...,[DE CONFORMIDAD con el art. 33 del Convenio de...,[],[]
2,"[Organización de la Administración, Sistema tr...",2021-02-18,"[Resolución de 10 de febrero de 2021, de la Se...",[BOE-A-2021-2457],"[Administración Local, Ayuntamientos, Cesión d...",[BOE-A-2020-17339],[DE CONFORMIDAD con],"[el art. 95.3 de la Ley 11/2020, de 30 de dici...",[DE CONFORMIDAD con el art. 95.3 de la Ley 11/...,[Ley 11/2020],[]
3,"[Relaciones internacionales, Turismo]",2021-02-18,[Acuerdo Internacional Administrativo entre la...,[BOE-A-2021-2456],"[Acuerdos internacionales, Congresos, Madrid, ...",[BOE-A-2015-8565],[DE CONFORMIDAD con],[el Convenio de 25 de junio de 2015],[DE CONFORMIDAD con el Convenio de 25 de junio...,[],[]
4,"[Asuntos sociales, Derecho Administrativo]",2021-02-18,[Corrección de errores del Real Decreto-ley 3/...,[BOE-A-2021-2453],"[Asistencia social, Ayudas, Registros administ...","[BOE-A-2021-1529, BOE-A-2020-5493]","[CORRECCIÓN de errores, CORRIGE errores]","[del Real Decreto-ley 3/2021, de 2 de febrero,...",[CORRECCIÓN de errores del Real Decreto-ley 3/...,"[Real Decreto-ley 3/2021, Real Decreto-ley 20/...",[]
5,[],2021-02-18,"[Orden SND/133/2021, de 17 de febrero, sobre l...",[BOE-A-2021-2458],[],"[BOE-A-2011-15623, BOE-A-1986-10498, BOE-A-202...","[DE CONFORMIDAD con, DE CONFORMIDAD con, EN RE...","[el art. 52 de la Ley 33/2011, de 4 de octubre...",[DE CONFORMIDAD con el art. 52 de la Ley 33/20...,"[Real Decreto 1418/1986, Real Decreto 926/2020...",[]


In [25]:
texto = ''
for refs in tabla_analisis['Referencias_completas']:
    for ref in refs:
        texto = texto + ref + ' '
        print(ref)

DE CONFORMIDAD con el art. 33 del Convenio de 25 de junio de 2015
DE CONFORMIDAD con al art. 3 de los Estatutos de 27 de septiembre de 1970
DE CONFORMIDAD con el art. 95.3 de la Ley 11/2020, de 30 de diciembre
DE CONFORMIDAD con el Convenio de 25 de junio de 2015
CORRECCIÓN de errores del Real Decreto-ley 3/2021, de 2 de febrero
CORRIGE errores en la disposición transitoria 8 del Real Decreto-ley 20/2020, de 29 de mayo
DE CONFORMIDAD con el art. 52 de la Ley 33/2011, de 4 de octubre
DE CONFORMIDAD con los arts. 2 y 3 de la Ley Orgánica 3/1986, de 14 de abri
EN RELACIÓN con la Orden PCM/79/2021, de 2 de febrero
CITA Real Decreto 926/2020, de 25 de octubre
CITA Ley 29/1998, de 13 de julio
CITA Real Decreto 1418/1986, de 13 de junio


In [26]:
texto = ''
for i, row in tabla_analisis.iterrows():
    #unique_id = i
    print('\n' + str(row['Item_id']) + str(row['Item_Name']) +'\n')

    antecedente = 1
    for ref in row['Referencias_completas']:
        texto = texto + ref + ' '
        print('\t' + 'Antecedente ' + str(antecedente) + ': ' + ref + ' ' + str(re.findall('|'.join(pattern), ref, flags=re.IGNORECASE)))
        antecedente += 1


['BOE-A-2021-2454']['Acta de la VIII Reunión de la Comisión Mixta Hispano-Hondureña de Cooperación por el que se aprueba el Marco de Asociación País España-Honduras 2020-2023, hecho en Madrid y Tegucigalpa el 16 de noviembre de 2020.']


['BOE-A-2021-2455']['Acuerdo entre la Organización Mundial del Turismo y el Ministerio de Industria, Comercio y Turismo del Reino de España, para el desarrollo de los ODS, reto demográfico y turismo rural, hecho en Madrid el 16 de diciembre de 2020.']

	Antecedente 1: DE CONFORMIDAD con el art. 33 del Convenio de 25 de junio de 2015 []
	Antecedente 2: DE CONFORMIDAD con al art. 3 de los Estatutos de 27 de septiembre de 1970 []

['BOE-A-2021-2457']['Resolución de 10 de febrero de 2021, de la Secretaría General de Financiación Autonómica y Local, por la que se desarrolla la información a suministrar por las Corporaciones locales relativa al esfuerzo fiscal de 2019 y su comprobación en las Delegaciones de Economía y Hacienda.']

	Antecedente 1: DE CONFOR

In [27]:
# Aplica expresiones REGEX para búsqueda de leyes, decretos, etc. referenciadas anteriormente
regex_result = re.findall('|'.join(pattern), texto, flags=re.IGNORECASE)

In [28]:
print(regex_result)

['Ley 11/2020', 'Real Decreto-ley 3/2021', 'Real Decreto-ley 20/2020', 'Ley 33/2011', 'Orden PCM/79/2021', 'Real Decreto 926/2020', 'Ley 29/1998', 'Real Decreto 1418/1986']


In [29]:
## Elimina duplicados
boletin_flat_list = list(set(regex_result))

In [30]:
boletin_flat_list

['Real Decreto 1418/1986',
 'Real Decreto-ley 3/2021',
 'Real Decreto 926/2020',
 'Ley 29/1998',
 'Ley 11/2020',
 'Ley 33/2011',
 'Orden PCM/79/2021',
 'Real Decreto-ley 20/2020']

## Importa BBDD ASECORP

In [31]:
ASECORP_BBDD = pd.read_csv('./ASECORP/ExportNormes_20210126.csv', delimiter=';')

In [32]:
ASECORP_BBDD

,Codigo,Numero,Titulo,CodigoAmbito,Ambito,Municipio,TiposNorma,IdTiposNorma
0,RES SLT/2313/2020 CAT,2313,"Resolución SLT/2313/2020, de 24 de septiembre,...",CAT,Cataluña,NaN,Prevención,[2]
1,(zzz) RES 12/4/2007 AST,12/4/2007,"Resolución de 12 abril de 2007, de la Consejer...",AST,Asturias,NaN,M. Ambiente,[1]
2,A CORUÑA 10/7/2015,10/7/2015,Modificación de la Ordenanza de vertidos y ser...,GAL,Galicia,A Coruña,M. Ambiente,[1]
3,A CORUÑA 11/8/2014 (1),11/8/2014,Ordenanza municipal reguladora de los procedim...,GAL,Galicia,A Coruña,M. Ambiente,[1]
4,A CORUÑA 11/8/2014 (2),11/8/2014,Ordenanza de protección contra la contaminació...,GAL,Galicia,A Coruña,M. Ambiente,[1]
...,...,...,...,...,...,...,...,...
19152,ZUMAIA 1/3/2001,1/3/2001,Ordenanza reguladora de la gestión del ciclo i...,PVA,Pais Vasco,Zumaia,M. Ambiente,[1]
19153,ZUMAIA 12/11/1999,12/11/1999,Normativa complementaria de las Normas Subsidi...,PVA,Pais Vasco,Zumaia,M. Ambiente,[1]
19154,ZUMAIA 15/6/2012,15/6/2012,Modificación de la Ordenanza municipal de la g...,PVA,Pais Vasco,Zumaia,M. Ambiente,[1]
19155,ZUMAIA 6/2/2004,6/2/2004,Modificación de la Normativa complementaria de...,PVA,Pais Vasco,Zumaia,M. Ambiente,[1]


In [33]:
ASECORP_BBDD_BOE = ASECORP_BBDD.loc[ASECORP_BBDD['Ambito'] == 'España']

In [34]:
ASECORP_BBDD_BOE

,Codigo,Numero,Titulo,CodigoAmbito,Ambito,Municipio,TiposNorma,IdTiposNorma
23,ACU 1/2020 ESP,1,Acuerdo Multilateral RID 1/2020 en virtud de l...,ESP,España,NaN,M. Ambiente / Prevención,[1][2]
35,ACU 11/6/2007 ESP,11/6/2007,Entrada en vigor del Acuerdo entre el Reino de...,ESP,España,NaN,M. Ambiente,[1]
57,ACU 17/10/2006 ESP,17/10/2006,Aplicación provisional del Acuerdo entre el Re...,ESP,España,NaN,M. Ambiente,[1]
71,ACU 19/11/2018 ESP,19/11/2018,Acuerdo de la Junta de Gobierno de la Confeder...,ESP,España,NaN,M. Ambiente,[1]
82,ACU 2/2020 ESP,2,Acuerdo Multilateral RID 2/2020 en virtud de l...,ESP,España,NaN,M. Ambiente / Prevención,[1][2]
...,...,...,...,...,...,...,...,...
18496,STC 86/2019 ESP,86,"Sentencia 86/2019, de 20 de junio de 2019. Rec...",ESP,España,NaN,M. Ambiente,[1]
18497,STC 87/2019 ESP,87,"Sentencia 87/2019, de 20 de junio de 2019. Rec...",ESP,España,NaN,M. Ambiente,[1]
18498,STC 88/2018 ESP,88,"Sentencia 88/2018, de 19 de julio de 2018. Con...",ESP,España,NaN,M. Ambiente / Reg. Industrial,[1][3]
18499,STC 9/3/2010 ESP,9/3/2010,"Sentencia de 9 de marzo de 2010, de la Sala Te...",ESP,España,NaN,M. Ambiente,[1]


In [35]:
# Crea nueva columna vacía de tipo lista en ASECORP_BBDD
ASECORP_BBDD_BOE['Tags'] = [[] for i in range(len(ASECORP_BBDD_BOE))]

In [36]:
titulo = []
for i, row in ASECORP_BBDD_BOE.iterrows():
    titulo.append(str(row['Titulo']) + str(re.findall('|'.join(pattern), str(row['Titulo']), flags=re.IGNORECASE)))
    ASECORP_BBDD_BOE['Tags'][i] = re.findall('|'.join(pattern), str(row['Titulo']), flags=re.IGNORECASE)

In [37]:
ASECORP_BBDD_BOE[1200:1250]

,Codigo,Numero,Titulo,CodigoAmbito,Ambito,Municipio,TiposNorma,IdTiposNorma,Tags
11629,ORD PRE/2827/2009 ESP,2827,"Orden PRE/2827/2009, de 19 de octubre, por la ...",ESP,España,NaN,M. Ambiente,[1],"[Orden PRE/2827/2009, Real Decreto 1370/2006]"
11630,ORD PRE/2843/2009 ESP,2843,"Orden PRE/2843/2009, de 19 de octubre, por la ...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/2843/2009, Real Decreto 2163/1994]"
11631,ORD PRE/2851/2010 ESP,2851,"Orden PRE/2851/2010, de 4 de noviembre, por la...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/2851/2010, Real Decreto 2163/1994]"
11632,ORD PRE/2871/2011 ESP,2871,"Orden PRE/2871/2011, de 25 de octubre, por la ...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/2871/2011, Real Decreto 2163/1994]"
11633,ORD PRE/2872/2011 ESP,2872,"Orden PRE/2872/2011, de 25 de octubre, por la ...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/2872/2011, Real Decreto 2163/1994]"
11634,ORD PRE/29/2004 ESP,29,"Orden PRE/29/2004, de 15 de enero, por la que ...",ESP,España,NaN,M. Ambiente,[1],"[Orden PRE/29/2004, Real Decreto 837/2002]"
11635,ORD PRE/2922/2005 ESP,2922,"Orden PRE/2922/2005, de 19 de septiembre, por ...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/2922/2005, Orden de 8 de marzo de 1..."
11636,ORD PRE/2957/2008 ESP,2957,"Orden PRE/2957/2008, de 10 de octubre, por la ...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/2957/2008, Real Decreto 2163/1994]"
11637,ORD PRE/3/2006 ESP,3,"Orden PRE/3/2006, de 12 de enero, por la que s...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/3/2006, Real Decreto 255/2003]"
11638,ORD PRE/3159/2004 ESP,3159,"Orden PRE/3159/2004, de 28 de septiembre, por ...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/3159/2004, Real Decreto 1406/1989]"


In [38]:
# titulo

In [39]:
len(titulo)

2959

In [40]:
## Busca patrones regex definidos en columna títulos del DF 
boletin = []
for i, row in ASECORP_BBDD_BOE.iterrows():
    boletin.append(str(re.findall('|'.join(pattern), row['Titulo'], flags=re.IGNORECASE)))

In [41]:
boletin[0:25]

['[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 "['Acuerdo Multilateral M-150']",
 "['Acuerdo Multilateral M-159']",
 "['Acuerdo multilateral M-168']",
 "['Acuerdo Multilateral M-170']",
 "['Acuerdo Multilateral M-171']",
 "['Acuerdo Multilateral M-173']",
 "['Acuerdo Multilateral M-175']",
 '[]',
 "['Acuerdo Multilateral M-178']",
 "['Acuerdo Multilateral M-198']"]

In [42]:
## Para cada fila de la BBDD recoge la expresión REGEX encontrada y si no existe no la incluye 
## en la lista resultante llamada 'boletin'. Además si no detecta expresión la cuenta como vacía
## en variable 'n_vacios', y la añade a la lista 'vacios' para inspeccionar posteriormente 
boletin = []
vacios = []
n_vacios = 0
for i, row in ASECORP_BBDD_BOE.iterrows():
    regex_result = re.findall('|'.join(pattern), row['Titulo'], flags=re.IGNORECASE)
    if len(regex_result) != 0:
        boletin.append(re.findall('|'.join(pattern), row['Titulo'], flags=re.IGNORECASE))
    else:
        n_vacios += 1
        vacios.append(row['Titulo'])

In [43]:
n_vacios

276

In [44]:
vacios[0:25]

['Acuerdo Multilateral RID 1/2020 en virtud de la Sección 1.5.1 del Reglamento del Transporte Internacional de Mercancías Peligrosas por Ferrocarril (RID), relativo a los certificados de consejeros de seguridad de conformidad con el punto 1.8.3.7 del RID, hecho en Madrid el 25 de marzo de 2020',
 'Entrada en vigor del Acuerdo entre el Reino de España y el Principado de Andorra sobre traslado y gestión de residuos, hecho en Madrid el 17 de octubre de 2006.',
 'Aplicación provisional del Acuerdo entre el Reino de España y el Principado de Andorra sobre traslado y gestión de residuos, hecho en Madrid el 17 de octubre de 2006.',
 'Acuerdo de la Junta de Gobierno de la Confederación Hidrográfica del Segura celebrada el día 30 de octubre de 2018 sobre aplicación de indisponibilidad o reducción de dotaciones de acuerdo al Plan Especial de Sequía y al Real Decreto que la regula',
 'Acuerdo Multilateral RID 2/2020 en virtud de la Sección 1.5.1 del Reglamento del Transporte Internacional de Merc

In [45]:
boletin[0:25]

[['Acuerdo Multilateral M-150'],
 ['Acuerdo Multilateral M-159'],
 ['Acuerdo multilateral M-168'],
 ['Acuerdo Multilateral M-170'],
 ['Acuerdo Multilateral M-171'],
 ['Acuerdo Multilateral M-173'],
 ['Acuerdo Multilateral M-175'],
 ['Acuerdo Multilateral M-178'],
 ['Acuerdo Multilateral M-198'],
 ['Acuerdo Multilateral M-214'],
 ['Acuerdo Multilateral M-218'],
 ['Acuerdo Multilateral M-226'],
 ['Acuerdo Multilateral M-228'],
 ['Acuerdo Multilateral M-231'],
 ['Acuerdo Multilateral M-236'],
 ['Acuerdo Multilateral M-245'],
 ['Acuerdo Multilateral M-254'],
 ['Acuerdo Multilateral M-259'],
 ['Acuerdo Multilateral M-265'],
 ['Acuerdo Multilateral M-268'],
 ['Acuerdo Multilateral M-271'],
 ['Acuerdo Multilateral M-273'],
 ['Acuerdo Multilateral M-276'],
 ['Acuerdo Multilateral M-291'],
 ['Acuerdo Multilateral M-292']]

In [46]:
# Flatten list of lists
boletin_ASECORP_flat_list = [item for sublist in boletin for item in sublist]

In [47]:
boletin_ASECORP_flat_list[0:25]

['Acuerdo Multilateral M-150',
 'Acuerdo Multilateral M-159',
 'Acuerdo multilateral M-168',
 'Acuerdo Multilateral M-170',
 'Acuerdo Multilateral M-171',
 'Acuerdo Multilateral M-173',
 'Acuerdo Multilateral M-175',
 'Acuerdo Multilateral M-178',
 'Acuerdo Multilateral M-198',
 'Acuerdo Multilateral M-214',
 'Acuerdo Multilateral M-218',
 'Acuerdo Multilateral M-226',
 'Acuerdo Multilateral M-228',
 'Acuerdo Multilateral M-231',
 'Acuerdo Multilateral M-236',
 'Acuerdo Multilateral M-245',
 'Acuerdo Multilateral M-254',
 'Acuerdo Multilateral M-259',
 'Acuerdo Multilateral M-265',
 'Acuerdo Multilateral M-268',
 'Acuerdo Multilateral M-271',
 'Acuerdo Multilateral M-273',
 'Acuerdo Multilateral M-276',
 'Acuerdo Multilateral M-291',
 'Acuerdo Multilateral M-292']

In [48]:
## Elimina duplicados
boletin_ASECORP_flat_list = list(set(boletin_ASECORP_flat_list))

In [49]:
boletin_ASECORP_flat_list[0:25]

['Orden ITC/1522/2007',
 'Orden AAA/1601/2012',
 'Orden AAA/1072/2013',
 'Resolución de 16 de marzo de 2020',
 'Resolución de 27 de diciembre de 2013',
 'Real Decreto 210/2004',
 'Real Decreto 1315/1992',
 'Orden INT/2323/2011',
 'Real Decreto 308/1983',
 'Orden de 15 de octubre de 2001',
 'Orden SSI/890/2017',
 'Nota de servicio 1/2016',
 'Orden de 20 de noviembre de 2001',
 'Real Decreto 736/2020',
 'Real decreto 1523/1999',
 'Resolución de 16 de abril de 2012',
 'Orden de 3 de septiembre de 1998',
 'Real Decreto 195/2010',
 'Resolución de 17 de abril de 2009',
 'Real Decreto 949/1997',
 'Real Decreto 1359/2011',
 'Real Decreto 1751/1998',
 'Resolución de 9 de junio de 2008',
 'Resolución de 21 de mayo de 2009',
 'Orden de 23 de diciembre de 1994']

In [50]:
## Busca coincidencias entre lista boletines BOEs explorados y lista boletines de BBDD ASECORP
set(boletin_flat_list) & set(boletin_ASECORP_flat_list)

{'Ley 11/2020', 'Ley 33/2011'}

In [51]:
#tabla_analisis['Tags'].isin(ASECORP_BBDD_BOE['Tags'])
for row_to_compare in tabla_analisis['Tags']:
    for row_comparing in ASECORP_BBDD_BOE['Tags']:
        if set(row_comparing) & set(row_to_compare):
            print(set(row_comparing) & set(row_to_compare))

{'Ley 11/2020'}
{'Ley 33/2011'}


In [52]:
for i, row_to_compare in tabla_analisis.iterrows():
    for j, row_comparing in ASECORP_BBDD_BOE.iterrows():
        if set(row_to_compare['Tags']) & set(row_comparing['Tags']):
            tabla_analisis['Match_ASECORP_BBDD'][i].append (ASECORP_BBDD_BOE['Codigo'][j])
            print(str(set(row_to_compare['Tags']) & set(row_comparing['Tags'])) + ' ' + str(row_comparing['Codigo']))

{'Ley 11/2020'} LEY 11/2020 ESP
{'Ley 33/2011'} LEY 33/2011 ESP


In [53]:
tabla_analisis

,Alertas,Fecha_publicacion,Item_Name,Item_id,Materias,Referencias,Referencias_palabra,Referencias_texto,Referencias_completas,Tags,Match_ASECORP_BBDD
0,[Relaciones internacionales],2021-02-18,[Acta de la VIII Reunión de la Comisión Mixta ...,[BOE-A-2021-2454],"[Acuerdos internacionales, Ayuda al desarrollo...",[],[],[],[],[],[]
1,"[Relaciones internacionales, Turismo]",2021-02-18,[Acuerdo entre la Organización Mundial del Tur...,[BOE-A-2021-2455],"[Acuerdos internacionales, Organización Mundia...","[BOE-A-2015-8565, BOE-A-1974-1937]","[DE CONFORMIDAD con, DE CONFORMIDAD con]",[el art. 33 del Convenio de 25 de junio de 201...,[DE CONFORMIDAD con el art. 33 del Convenio de...,[],[]
2,"[Organización de la Administración, Sistema tr...",2021-02-18,"[Resolución de 10 de febrero de 2021, de la Se...",[BOE-A-2021-2457],"[Administración Local, Ayuntamientos, Cesión d...",[BOE-A-2020-17339],[DE CONFORMIDAD con],"[el art. 95.3 de la Ley 11/2020, de 30 de dici...",[DE CONFORMIDAD con el art. 95.3 de la Ley 11/...,[Ley 11/2020],[LEY 11/2020 ESP]
3,"[Relaciones internacionales, Turismo]",2021-02-18,[Acuerdo Internacional Administrativo entre la...,[BOE-A-2021-2456],"[Acuerdos internacionales, Congresos, Madrid, ...",[BOE-A-2015-8565],[DE CONFORMIDAD con],[el Convenio de 25 de junio de 2015],[DE CONFORMIDAD con el Convenio de 25 de junio...,[],[]
4,"[Asuntos sociales, Derecho Administrativo]",2021-02-18,[Corrección de errores del Real Decreto-ley 3/...,[BOE-A-2021-2453],"[Asistencia social, Ayudas, Registros administ...","[BOE-A-2021-1529, BOE-A-2020-5493]","[CORRECCIÓN de errores, CORRIGE errores]","[del Real Decreto-ley 3/2021, de 2 de febrero,...",[CORRECCIÓN de errores del Real Decreto-ley 3/...,"[Real Decreto-ley 3/2021, Real Decreto-ley 20/...",[]
5,[],2021-02-18,"[Orden SND/133/2021, de 17 de febrero, sobre l...",[BOE-A-2021-2458],[],"[BOE-A-2011-15623, BOE-A-1986-10498, BOE-A-202...","[DE CONFORMIDAD con, DE CONFORMIDAD con, EN RE...","[el art. 52 de la Ley 33/2011, de 4 de octubre...",[DE CONFORMIDAD con el art. 52 de la Ley 33/20...,"[Real Decreto 1418/1986, Real Decreto 926/2020...",[LEY 33/2011 ESP]


In [54]:
tabla_analisis['Match_ASECORP_BBDD'][1]

[]

In [55]:
tabla_analisis['Tags'][1]

[]

# Genera Fichero EXCEL de resultados

In [56]:
## Cambia orden de columnas y elimina las no necesarias  
tabla_analisis_final = tabla_analisis[['Item_id','Item_Name','Fecha_publicacion','Materias','Alertas','Referencias','Referencias_completas','Tags','Match_ASECORP_BBDD']]

In [57]:
## Crea función que convierte lista a string en todas las columnas de tabla_analisis
## para evitar en presentación final los caracteres [' '] propios de las listas
def list2Str(lst):
    if type(lst) is list: # apply conversion to list columns
        return", ".join(lst)
    else:
        return lst

In [58]:
## Aplica función de conversión de listas a strings
tabla_analisis_final = tabla_analisis_final.apply(lambda x: [list2Str(i) for i in x])

In [59]:
  tabla_analisis_final[0:10]

,Item_id,Item_Name,Fecha_publicacion,Materias,Alertas,Referencias,Referencias_completas,Tags,Match_ASECORP_BBDD
0,BOE-A-2021-2454,Acta de la VIII Reunión de la Comisión Mixta H...,2021-02-18,"Acuerdos internacionales, Ayuda al desarrollo,...",Relaciones internacionales,,,,
1,BOE-A-2021-2455,Acuerdo entre la Organización Mundial del Turi...,2021-02-18,"Acuerdos internacionales, Organización Mundial...","Relaciones internacionales, Turismo","BOE-A-2015-8565, BOE-A-1974-1937",DE CONFORMIDAD con el art. 33 del Convenio de ...,,
2,BOE-A-2021-2457,"Resolución de 10 de febrero de 2021, de la Sec...",2021-02-18,"Administración Local, Ayuntamientos, Cesión de...","Organización de la Administración, Sistema tri...",BOE-A-2020-17339,DE CONFORMIDAD con el art. 95.3 de la Ley 11/2...,Ley 11/2020,LEY 11/2020 ESP
3,BOE-A-2021-2456,Acuerdo Internacional Administrativo entre la ...,2021-02-18,"Acuerdos internacionales, Congresos, Madrid, O...","Relaciones internacionales, Turismo",BOE-A-2015-8565,DE CONFORMIDAD con el Convenio de 25 de junio ...,,
4,BOE-A-2021-2453,Corrección de errores del Real Decreto-ley 3/2...,2021-02-18,"Asistencia social, Ayudas, Registros administr...","Asuntos sociales, Derecho Administrativo","BOE-A-2021-1529, BOE-A-2020-5493",CORRECCIÓN de errores del Real Decreto-ley 3/2...,"Real Decreto-ley 3/2021, Real Decreto-ley 20/2020",
5,BOE-A-2021-2458,"Orden SND/133/2021, de 17 de febrero, sobre la...",2021-02-18,,,"BOE-A-2011-15623, BOE-A-1986-10498, BOE-A-2021...",DE CONFORMIDAD con el art. 52 de la Ley 33/201...,"Real Decreto 1418/1986, Real Decreto 926/2020,...",LEY 33/2011 ESP


In [60]:
## Generar hyperlink a artículo BOE en CSV "=HYPERLINK("https://www.boe.es/boe/dias/2021/02/02/pdfs/BOE-A-2021-1474.pdf";"BOE-A-2021-1474")"
## https://www.boe.es/diario_boe/xml.php?id=## https://www.boe.es/boe/dias/2021/02/02/pdfs/BOE-A-2021-1474.pdf

tabla_analisis_final_CSV = tabla_analisis_final

In [61]:
# Compone y genera enlace a PDF del BOE correspondiente
tabla_analisis_final_CSV['Item_id'] = '=HIPERVINCULO(' + '"https://www.boe.es/boe/dias/' \
                                                        + tabla_analisis_final_CSV['Fecha_publicacion'].map(lambda x: x.strftime('%Y')) + '/'\
                                                        + tabla_analisis_final_CSV['Fecha_publicacion'].map(lambda x: x.strftime('%m')) + '/'\
                                                        + tabla_analisis_final_CSV['Fecha_publicacion'].map(lambda x: x.strftime('%d')) + '/'\
                                                        + 'pdfs/'\
                                                        + tabla_analisis_final_CSV['Item_id'] + '.pdf";'\
                                                        + '"' + tabla_analisis_final_CSV['Item_id'] + '")'

In [62]:
print(tabla_analisis_final_CSV['Item_id'][0])

=HIPERVINCULO("https://www.boe.es/boe/dias/2021/02/18/pdfs/BOE-A-2021-2454.pdf";"BOE-A-2021-2454")


In [63]:
# tabla_analisis_final_CSV.to_excel("./ASECORP/Resultados_Matching.xlsx", sheet_name='MATCHING_BOE') 

In [64]:
tabla_analisis_final_CSV.to_csv("./ASECORP/Resultados_Matching_BOE_" + today.strftime("%Y%m%d") + ".csv", index=False) 

# ------------------------------------------------------------------------

# DOGC

In [65]:
URL_HTML_resumen =  "https://dogc.gencat.cat/es/index.html?newLang=es_ES&language=es_ES"

In [66]:
from requests_html import HTMLSession
session = HTMLSession()
r = session.get(URL_HTML_resumen)

In [67]:
r.html.links

{'/ca/inici/index.html',
 '/ca/serveis/dogc-republica-i-exili/',
 '/ca/serveis/publicar-un-anunci/',
 '/es/ajuda/mapaweb/',
 '/es/index.html?newLang=es_ES&language=es_ES',
 '/es/inici/',
 '/es/inici/resultats/index.html?orderBy=3&page=1&typeSearch=1&advanced=true&current=true&title=false&numResultsByPage=50&publicationDateInitial=18/11/2020&value=%22anunci%20d%27informaci%C3%B3%20p%C3%BAblica%22%20OR%20%22sotmet%20a%20informaci%C3%B3%20p%C3%BAblica%22%20OR%20%22sotmeten%20a%20informaci%C3%B3%20p%C3%BAblica%22%20OR%20%22sobre%20informaci%C3%B3%20p%C3%BAblica%22%20OR%20%22aprovaci%C3%B3%20inicial%22%20OR%20%22anuncio%20de%20informaci%C3%B3n%20p%C3%BAblica%22%20OR%20%22somete%20a%20informaci%C3%B3n%20p%C3%BAblica%22%20OR%20%22someten%20a%20informaci%C3%B3n%20p%C3%BAblica%22%20OR%20%22sobre%20informaci%C3%B3n%20p%C3%BAblica%22%20OR%20%22aprobaci%C3%B3n%20inicial%22',
 '/es/inici/resultats/index.html?orderBy=3&page=1&typeSearch=1&advanced=true&current=true&title=true&numResultsByPage=50&pub

In [68]:
r.html.xpath('//*[@id="titol"]/span/text()')

[]

In [69]:
r.html.xpath('//*[@id="titol"]/h3/text()')


[]

In [70]:
r.html.lxml.tag

'html'

In [71]:
r.html.lxml.xpath('//*[@id="titol"]/span')

[]

In [72]:
#import requests
#from lxml import html

response = requests.get(URL_HTML_resumen)

tree = html.fromstring(response.text)

# fetches all useful elements 
useful = tree.xpath('//*[@id="titol"]/span/text()')

print(useful)


[]


In [73]:
r.html.lxml.xpath('//*[@id="sumari"]/ul/li[1]/form/@action')

[]

In [74]:
r.html.lxml.xpath('//*[@id="sumari"]/ul/li[1]/form/input[1]/@name')

[]

In [75]:
r.html.lxml.xpath('//*[@id="sumari"]/ul/li[1]/form/input[1]/@value')

[]

In [76]:
r.html.lxml.xpath('//*[@id="sumari"]/ul/li[1]/form')

[]

In [77]:
useful = tree.xpath('//*[@id="sumari"]/ul/li[1]/form/input[3]/@value')

print(useful)

[]


In [78]:
tree.xpath('//*[@id="sumari"]/ul/li[1]/form/@action')

[]

In [79]:
cadena = ''
for input in tree.xpath('//*[@id="sumari"]/ul/li[1]/form'):
    argumentos = input.xpath('./input/@name')
    valores = input.xpath('./input/@value')
    print(argumentos, valores)

for indice in range(len(argumentos)):
    cadena += argumentos[indice] + '=' + valores[indice] + '&'

argumentos[1]
cadena[:-1]

NameError: name 'argumentos' is not defined

## Recoge Nombre Secciones Sumario

In [80]:
import re
seccion = tree.xpath('//*[@id="sumari"]/ul/li[1]/form/a/text()')
seccion = re.sub('(\\r|\\n|\\t|\ )+', '', seccion[0])
seccion

IndexError: list index out of range

## Recoge Nombre Secciones Sumario

In [ ]:
secciones = tree.xpath('//*[@id="sumari"]/ul/li')
#print(secciones)
df_secciones_sumarios = pd.DataFrame()
for seccion in secciones:
    seccion = seccion.xpath('./form/a/text()')
    seccion = re.sub('(\\r|\\n|\\t)+', '', seccion[0])
    print(seccion[1:-1])
    df_secciones_sumarios = df_secciones_sumarios.append({'Seccion': seccion[1:-1]}, ignore_index=True)


In [ ]:
df_secciones_sumarios

## Recoge Valores para formar URLs Secciones Sumario

In [ ]:
URL_base_sumario = tree.xpath('//*[@id="sumari"]/ul/li[1]/form/@action')

df_URL_sumarios = pd.DataFrame()
for seccion in secciones:    
    cadena = ''
    for input in seccion.xpath('./form'):
        argumentos = input.xpath('./input/@name')
        valores = input.xpath('./input/@value')
        #print(argumentos, valores)

    for indice in range(len(argumentos)):
        cadena += argumentos[indice] + '=' + valores[indice] + '&'

    URL_sumario = 'https://dogc.gencat.cat' + str(URL_base_sumario[0]) + '?' + str(cadena[:-1])
    print(URL_sumario)
    df_URL_sumarios = df_URL_sumarios.append({'URL_Seccion': URL_sumario}, ignore_index=True)

In [ ]:
df_URL_sumarios

In [ ]:
 df_sumarios = pd.concat([df_secciones_sumarios, df_URL_sumarios], axis=1)

In [ ]:
df_sumarios

In [ ]:
df_sumarios['URL_Seccion'][4]

## Recoge Items en Seccion Disposiciones

In [ ]:
### Recoge Items en Seccion Disposiciones

response = requests.get(df_sumarios['URL_Seccion'][0])
sumario_HTML = html.fromstring(response.text)

#seccion = sumario_HTML.xpath('//*[@id="disposicions"]/div[1]/text()')
seccion = sumario_HTML.xpath('//*[@id="disposicions"]/div/text()')
bloques = sumario_HTML.xpath('//*[@id="disposicions"]')
df_disposiciones = pd.DataFrame(columns=['Seccion','Item_Name','PDF_link'])
for bloque in bloques: 
    item_name = bloque.xpath('./div/p/text()')
    #item_name = re.sub('(\\r|\\n|\\t)+', '', item_name[0])
    pdf_link = bloque.xpath('./div/div/a[4]/@href')

for row in range(len(pdf_link)):
    df_disposiciones = df_disposiciones.append({'Item_Name': re.sub('(\\r|\\n|\\t)+', '',                                                                               item_name[row]),
                                            'PDF_link' : pdf_link[row],
                                            'Seccion' : seccion[0][:-1]},
                                            ignore_index=True)

df_disposiciones

In [ ]:
pdf_link

# DOUE

In [81]:
# --------------------------------------------------------------------------------------
# DOUE
# --------------------------------------------------------------------------------------

URL_HTML_resumen =  "https://eur-lex.europa.eu/oj/direct-access.html?locale=es"

## carga página HTML y genera árbol

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
response = requests.get(URL_HTML_resumen, headers=headers)
tree = html.fromstring(response.text)

## Recoge Nombre Secciones Sumario

secciones = tree.xpath('//*[@class="table table-striped table-hover table-condensed OJTable AllBordersTable"]')
#print(secciones)


In [82]:
source_dir = './DOUEs'
target_dir = './DOUEs_Anteriores'
    
file_names = os.listdir(source_dir)
    
for file_name in file_names:
    #shutil.move(os.path.join(source_dir, file_name), target_dir)
    # Evita que de error si el fichero que se mueve ya existe en dir destino
    try:
        os.remove(os.path.join(target_dir, file_name))
        shutil.move(os.path.join(source_dir, file_name), target_dir)
    except OSError:
        shutil.move(os.path.join(source_dir, file_name), target_dir)

In [99]:
## Recoge Valores para formar URLs Secciones Sumario

df_secciones_sumarios = pd.DataFrame()
for seccion in secciones:
    seccion_name = seccion.xpath('./tbody/tr[1]/td[2]/a/text()')
    seccion_URL = seccion.xpath('./tbody/tr[1]/td[2]/a/@href')
    #seccion = re.sub('(\\r|\\n|\\t)+', '', seccion)
    print(seccion_name)
    #print(seccion_URL)


### Si no hay disposiciones hay que gestionar el error y salir del script
if len(seccion_name) == 0:
    print('No hay disposiciones')
    # exit()

for row in range(len(seccion_name)):
    df_secciones_sumarios = df_secciones_sumarios.append({'Seccion': seccion_name[row],
                                                            'Seccion_link' : 'https://eur-lex.europa.eu' + seccion_URL[row][4:]},
                                                            ignore_index=True)

[]
No hay disposiciones


In [93]:
if len(df_secciones_sumarios['Seccion'][0]) == 0:
    print('No hay disposiciones')

KeyError: 'Seccion'

In [84]:
df_secciones_sumarios

""


In [85]:
# df_secciones_sumarios = pd.DataFrame({'Seccion':['L049'], 'Seccion_link':['https://eur-lex.europa.eu/legal-content/ES/TXT/?uri=OJ:L:2021:049:TOC']})

In [86]:
[ print (row) for row in df_secciones_sumarios['Seccion_link']]

KeyError: 'Seccion_link'

In [ ]:
DOUE_sumarios = pd.DataFrame()

for URL in df_secciones_sumarios['Seccion_link']:
    
    ### Recoge Items en Seccion Legislación

    #print(URL)
    response = requests.get(URL)
    sumario_HTML = html.fromstring(response.text)

    documentos_Name = sumario_HTML.xpath('//*[@class="oj-ti-doc-dur"]/a[1]/text()[1]')
    documentos_URL = sumario_HTML.xpath('//*[@class="oj-ti-doc-dur"]/a[1]/@href')

    for row in range(len(documentos_Name)):
        DOUE_sumarios = DOUE_sumarios.append({'Seccion': 'L' + documentos_URL[row][-7:-4],
                                        'Item_Title': documentos_Name[row],
                                        'Item_Link': 'https://eur-lex.europa.eu' + documentos_URL[row][10:],
                                        #'Item_Tag': documentos_URL[row][-45:-29],
                                        'Item_Doc_Name': '',
                                        'Fecha_publicacion': '',
                                        'PDF_Link': '',
                                        'Tags': '',
                                        'Match_ASECORP_BBDD': ''},
                                        ignore_index=True)


In [ ]:
# documentos_URL[1][-45:-29]

In [ ]:
DOUE_sumarios

In [ ]:
print(DOUE_sumarios['Item_Link'][1])

In [ ]:
def save_txt(txt, path):
    with open(path, 'w') as f:
        f.write(txt)


In [ ]:
documentos_FileName = sumario_HTML.xpath('//*[@class="DocumentTitle pull-left"]/text()')

In [ ]:
row = 0
for link in DOUE_sumarios['Item_Link']:
    sumario_Text = []
    ### Recoge PDF link en Items Legislación

    response = requests.get(link)
    sumario_HTML = html.fromstring(response.text)

    PDF_Link = sumario_HTML.xpath('//*[@id="format_language_table_PDF_ES"]/@href')
    DOUE_sumarios['PDF_Link'][row] = 'https://eur-lex.europa.eu' + PDF_Link[0][10:]

    Item_Doc_Name = sumario_HTML.xpath('//*[@class="DocumentTitle pull-left"]/text()')[0][9:]
    DOUE_sumarios['Item_Doc_Name'][row] = Item_Doc_Name

    #Item_Fecha = str(sumario_HTML.xpath('//*[@class="oj-hd-date"]/text()'))
    #Item_Fecha = datetime.strptime(str(raiz.xpath('//*[@class="oj-hd-date"]/text()'))[0:-2],"%d.%m.%Y")
    Item_Fecha = re.findall('[0-9]+\.[0-9]+\.[0-9]+', str(sumario_HTML.xpath('//*[@class="oj-hd-date"]/text()')))
    DOUE_sumarios['Fecha_publicacion'][row] = datetime.strptime(str(Item_Fecha)[2:-2],"%d.%m.%Y").date()

    ### Recoge Texto de Items
    
    Item_Text = sumario_HTML.xpath('//*[@class="oj-normal"]/text()')
    
    #[ sumario_Text.append(re.sub('(\\r||\\n\\t|\\xa0)+', '', item)) for item in Item_Text ]
    [ sumario_Text.append(item) for item in Item_Text ]
   
    ### Sustituye caracteres 'n.' por 'n.º' ya que al importar de HTML se pierde el caracter º

    sumario = "".join(sumario_Text).replace(' n.',' n.º')
    print(sumario)
    ### Busca tags en texto

    DOUE_sumarios['Tags'][row] = list(set(re.findall('|'.join(pattern), sumario, flags=re.IGNORECASE)))

    ### Salva texto de Items en formato TXT

    save_txt(sumario, './DOUEs/Resumen-DOUE-' + Item_Doc_Name + '.txt')
    #print(sumario)
    #print('./DOUEs/Resumen-DOUE-' + Item_Doc_Name + '.txt')
    #print(Item_Doc_Name)

    #print(list(set(re.findall('|'.join(pattern), sumario, flags=re.IGNORECASE))))

    row += 1


In [ ]:
# Elimina Tags duplicados
for i, row in DOUE_sumarios.iterrows():
    DOUE_sumarios['Tags'][i] = list(set(DOUE_sumarios['Tags'][i]))
    print(DOUE_sumarios['Tags'][i])

In [ ]:
DOUE_sumarios

In [ ]:
### Ordena Columnas
DOUE_sumarios = DOUE_sumarios[['Seccion', 'Item_Doc_Name', 'Fecha_publicacion', 'PDF_Link', 'Item_Title', 'Item_Link', 'Tags', 'Match_ASECORP_BBDD']]
DOUE_sumarios

In [ ]:
### Crea lista plana de Tags sin duplicados
DOUE_flat_list = list(set([item for row in DOUE_sumarios['Tags'] for item in row ]))

In [ ]:
print (DOUE_flat_list)

In [ ]:
## Busca coincidencias entre lista boletines BOEs explorados y lista boletines de BBDD ASECORP
list(set(DOUE_flat_list) & set(boletin_ASECORP_flat_list))

In [ ]:
for i, row_to_compare in DOUE_sumarios.iterrows():
    for j, row_comparing in ASECORP_BBDD_BOE.iterrows():
        if set(row_to_compare['Tags']) & set(row_comparing['Tags']):
            DOUE_sumarios['Match_ASECORP_BBDD'][i].append (ASECORP_BBDD_BOE['Codigo'][j])
            print(str(set(row_to_compare['Tags']) & set(row_comparing['Tags'])) + ' ' + str(row_comparing['Codigo']))

In [ ]:
DOUE_sumarios

In [ ]:
## Aplica función de conversión de listas a strings
DOUE_sumarios = DOUE_sumarios.apply(lambda x: [list2Str(i) for i in x])

In [ ]:
DOUE_sumarios_CSV = pd.DataFrame()
DOUE_sumarios_CSV = DOUE_sumarios

In [ ]:
# Compone y genera enlace a PDF del DOUE correspondiente
DOUE_sumarios_CSV['PDF_Link'] = '=HIPERVINCULO(' + '"' + DOUE_sumarios['PDF_Link'] + '";' + '"' + "Doc: " + DOUE_sumarios['Item_Doc_Name'] + '")'

In [ ]:
DOUE_sumarios_CSV

In [ ]:
DOUE_sumarios_CSV['PDF_Link'][1]

In [ ]:
### Genera fichero CSV

DOUE_sumarios_CSV.to_csv("./ASECORP/Resultados_Matching_DOUE_" + today.strftime("%Y%m%d") + ".csv", index=False) 